In [1]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from tqdm import notebook
import pydicom
import zipfile
import shutil
import io
import gzip
import torch

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
params=json_to_parameters('config.json')

In [3]:
train_df=pd.read_csv(params.path.data+'train.csv')
train_df.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,1,0,0,1,0,0,0
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,1,0,0,1,0,0,0


## Dicom images to pickled numpy

In [ ]:
# Run This if you don't have enough space to unzip the data files
with zipfile.ZipFile(params.path.data+'train.zip', mode = 'r', allowZip64 = True) as file:
    l = file.namelist()
    p = Pool()
    for i,name in enumerate(notebook.tqdm(l)):
#             os.makedirs(os.path.dirname(params.path.train+name), exist_ok=True)
        with file.open(name) as zf:
            img_dicom=pydicom.read_file(io.BytesIO(zf.read()))
            img = img_dicom.pixel_array.astype(np.int32)+np.int32(img_dicom.RescaleIntercept)
            del img_dicom
        _=p.apply_async(gzip_pickle,(img,params.path.train+name.split('.')[0]+'.pkl'))
        if (i+1)%2000 ==0:
            p.close()
            p.join()
            del p
            gc.collect()
            p = Pool()

## Prepare full_train.csv

In [ ]:
# prepare 'dcm_data.csv'
# You must run this after unzipping train.zip
train_imgs = sorted( glob.glob(f"{train_dir}*/*/*"))
errored_files=[]
df=dicom_get_df_data(train_imgs,image_stats=True,errored_files=errored_files)
df.to_csv(params.path.data+'dicom_data.csv', index=False)
print(len(errored_files))
del df

In [ ]:
# prepare 'full_train.csv'
dicom_df=pd.read_csv(params.path.data+'dicom_data.csv',low_memory=False)
df=train_df.merge(dicom_df,how='left',left_on=['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID'],
                 right_on=['study_instance_uid','series_instance_uid','sop_instance_uid'])
gp=df[['StudyInstanceUID','SeriesInstanceUID','instance_number']].groupby(['SeriesInstanceUID'])
gp_slice=gp.instance_number.agg(min_slice='min',max_slice='max').reset_index()
df=df.merge(gp_slice,on='SeriesInstanceUID',how='left')
df['rel_slice']=(df.instance_number-df.min_slice)/(df.max_slice-df.min_slice)
df['z']=df.image_position_patient.str.split(',').str[-1].str.split(']').str[0].astype(float)
gp=df[['StudyInstanceUID','SeriesInstanceUID','z']].groupby(['SeriesInstanceUID'])
gp_slice=gp.z.agg(z_min='min',z_max='max').reset_index()
df=df.merge(gp_slice,on='SeriesInstanceUID',how='left')
df['rel_z']=(df.z-df.z_min)/1200
df=df.sort_values('z').reset_index(drop=True)
df['new_place']=df[['SeriesInstanceUID','SOPInstanceUID']].groupby('SeriesInstanceUID')['SOPInstanceUID'].cumcount()+1
gp=df[['StudyInstanceUID','SeriesInstanceUID','new_place']].groupby(['SeriesInstanceUID'])
gp_slice=gp.new_place.agg(new_min_slice='min',new_max_slice='max').reset_index()
df=df.merge(gp_slice,on='SeriesInstanceUID',how='left')
df['new_rel_slice']=(df.new_place-df.new_min_slice)/(df.new_max_slice-df.new_min_slice)
df.to_csv(params.path.data+'full_train.csv', index=False)
